In [3]:
from googleapiclient.discovery import build
import pandas as pd
import streamlit as st

## API Key Connection

In [4]:
def Connect_API():    
    Api_id="API key"
    Api_serviceName="Youtube"
    Api_version="v3"
    connect=build(Api_serviceName,Api_version,developerKey=Api_id)
    return connect
youtube=Connect_API()


## Channel Information

In [5]:
def get_channel_details(channel_id):
    request=youtube.channels().list(
                                part="brandingSettings,contentDetails,contentOwnerDetails,id,localizations,snippet,statistics,status,topicDetails",
                                id=channel_id
                                )
    response=request.execute()
    for i in response['items']:
        data={
            "Channel_Name": i['snippet']['title'],
            "Channel_Id": i["id"],
            "Subscription_Count": i['statistics']['subscriberCount'],
            "Channel_Views": i['statistics']['viewCount'],
            "Channel_Videos":i['statistics']['videoCount'],
            "Channel_Description": i['snippet']['description'],
            "Playlist_Id": i['contentDetails']['relatedPlaylists']['uploads']
        }
    return data
channel_details=get_channel_details('UC_HZY9d5wJ-MEiuq6vhx8hg')


In [6]:
channel_details


{'Channel_Name': 'JK ',
 'Channel_Id': 'UC_HZY9d5wJ-MEiuq6vhx8hg',
 'Subscription_Count': '888000',
 'Channel_Views': '104979422',
 'Channel_Videos': '234',
 'Channel_Description': '🤓\n',
 'Playlist_Id': 'UU_HZY9d5wJ-MEiuq6vhx8hg'}

## Using Playlist_Id for finding Video_ID

In [7]:
def get_videoIds(channel_id):    
    response=youtube.channels().list(
        part='contentDetails,snippet,statistics',
        id=channel_id
    )
    playlist_id=response.execute()
    playlist_ids=playlist_id['items'][0]['contentDetails']['relatedPlaylists']['uploads']
    
    video_ids=[]
    next_page_token=None
    while True:
        response1=youtube.playlistItems().list( part="snippet",
            playlistId=playlist_ids,
            maxResults=50,                                              #maxResults is used to get 1-50 videos.
            pageToken=next_page_token).execute()
       
        
        for i in range (len(response1['items'])):
            r=response1['items'][i]['snippet']['resourceId']['videoId']
            video_ids.append(r)
        next_page_token=response1.get('nextPageToken')
        if next_page_token==None:
            break
    return(video_ids)
videoIds=get_videoIds("UCuMXWW_kAUsW4dC1ZBNGzzQ")
len(videoIds)
    




KeyboardInterrupt: 

In [ ]:
len(videoIds)


284

## Get Video Details using video_id

In [ ]:
def get_videoDetails(videoIds):
    videos=[]   
    for video_id in videoIds:
        request1 = youtube.videos().list(
            part="contentDetails,snippet,statistics",
            id=video_id
        )
        r=request1.execute()
        for items in r['items']:
            data={
                "Channel_Name":items["snippet"]["channelTitle"],
                "Channel_Id":items['snippet']['channelId'],
            "Video_Id":items['id'],
            "Video_Name": items['snippet']['title'],
            "Video_Description":items['snippet']['description'],
            "Tags":items['snippet'].get('tags'),
            "PublishedAt": items['snippet']['publishedAt'],
            "View_Count":items['statistics']['viewCount'],
            "Like_Count": items['statistics'].get('likeCount'),
            "Dislike_Count": items.get("dislikesCount"),
            "Favorite_Count": items['statistics']['favoriteCount'],
            "Definition": items['contentDetails']['definition'],
            "Duration": items['contentDetails']['duration'],
            "Thumbnail":items['snippet']['thumbnails']['default']['url'],
            "Caption_Status": "Available"
            }
        videos.append(data)
    return videos
video_Details=get_videoDetails(videoIds)




In [ ]:
video_Details[0]


{'Channel_Name': 'Shree Naval Kishori',
 'Channel_Id': 'UCuMXWW_kAUsW4dC1ZBNGzzQ',
 'Video_Id': 'E0jcUhEj4AQ',
 'Video_Name': 'Shree Ram Chandra Kripalu Bhajman #shorts',
 'Video_Description': '',
 'Tags': None,
 'PublishedAt': '2024-11-24T02:35:20Z',
 'View_Count': '3713',
 'Like_Count': '1083',
 'Dislike_Count': None,
 'Favorite_Count': '0',
 'Definition': 'hd',
 'Duration': 'PT31S',
 'Thumbnail': 'https://i.ytimg.com/vi/E0jcUhEj4AQ/default.jpg',
 'Caption_Status': 'Available'}

## Get Comments using videoId and CommentThread()

In [ ]:

def comments_info(videoIds):
    comments=[]
    for video_id in videoIds:
        try:
            request = youtube.commentThreads().list(
                part='snippet,replies',
                videoId=video_id,
                maxResults=100
            )
            response = request.execute()

            for item in response['items']:
                data = {
                    "Video_id":video_id,
                    "Comment_Id": item['snippet']['topLevelComment']['id'],
                    "Comment_Text": item['snippet']['topLevelComment']['snippet']['textDisplay'],
                    "Comment_Author": item['snippet']['topLevelComment']['snippet']['authorDisplayName'],
                    "Comment_PublishedAt": item['snippet']['topLevelComment']['snippet']['publishedAt']
                }
                comments.append(data)

        except:
            pass
    return comments
commentsInfo=comments_info(videoIds)
            





In [ ]:
commentsInfo

[{'Video_id': 'E0jcUhEj4AQ',
  'Comment_Id': 'UgxxXPTnp99KXgMuJ8F4AaABAg',
  'Comment_Text': 'Hare Rama hare hare',
  'Comment_Author': '@RathnakaranCk-p7y',
  'Comment_PublishedAt': '2024-11-24T06:25:25Z'},
 {'Video_id': 'E0jcUhEj4AQ',
  'Comment_Id': 'UgwTAfY8gl68I0_vj7x4AaABAg',
  'Comment_Text': 'Jai shree Ram 🏹🚩🙏🌸♥️❤️❤️',
  'Comment_Author': '@PriyankaKumari-ir2en',
  'Comment_PublishedAt': '2024-11-24T06:23:07Z'},
 {'Video_id': 'E0jcUhEj4AQ',
  'Comment_Id': 'UgxbZRJetTSNQtAoyPB4AaABAg',
  'Comment_Text': '🙏👌👌👌👌',
  'Comment_Author': '@AnilKusekar-z4j',
  'Comment_PublishedAt': '2024-11-24T06:01:58Z'},
 {'Video_id': 'E0jcUhEj4AQ',
  'Comment_Id': 'UgwlMguMFlTsG7vGqfR4AaABAg',
  'Comment_Text': 'Didi ai kon hai....? 😢',
  'Comment_Author': '@sujoykarmakar3316',
  'Comment_PublishedAt': '2024-11-24T05:53:08Z'},
 {'Video_id': 'E0jcUhEj4AQ',
  'Comment_Id': 'UgwFX1sYSKycEnclKIh4AaABAg',
  'Comment_Text': '❤❤❤❤',
  'Comment_Author': '@mohankrishna5952',
  'Comment_PublishedAt': '2024-

## Get Playlist Details


In [ ]:
def PlaylistDetails(channel_id):
    next_page_token=None
    playlist_details=[]
    while True:    
        request=youtube.playlists().list(
                        part="snippet,contentDetails",
                        channelId=channel_id,
                        maxResults=50,
                        pageToken=next_page_token)
        response=request.execute()
        for i in  response['items']:
            data={"Channel_Id":i['snippet']['channelId'],
                  "Channel_Name":i['snippet']['channelTitle'],
                "Playlist_Id":i['id'],
                "Published_At":i['snippet']['publishedAt'],
                "Video_Count":i['contentDetails']['itemCount']
                }
            playlist_details.append(data)
        next_page_token=response.get('nextPageToken')    
        if next_page_token==None:
            break
    return playlist_details
Playlist_Details=PlaylistDetails("UCuI5XcJYynHa5k_lqDzAgwQ")

In [ ]:
Playlist_Details

[{'Channel_Id': 'UCuI5XcJYynHa5k_lqDzAgwQ',
  'Channel_Name': 'Data Science Tamil',
  'Playlist_Id': 'PLuo9105K6NLRjdlnTk8fLAqll3fiKP6aq',
  'Published_At': '2024-11-10T12:38:24.320848Z',
  'Video_Count': 4},
 {'Channel_Id': 'UCuI5XcJYynHa5k_lqDzAgwQ',
  'Channel_Name': 'Data Science Tamil',
  'Playlist_Id': 'PLuo9105K6NLTva5CuB0_JGlhabrc9B96U',
  'Published_At': '2024-11-01T09:55:26Z',
  'Video_Count': 10},
 {'Channel_Id': 'UCuI5XcJYynHa5k_lqDzAgwQ',
  'Channel_Name': 'Data Science Tamil',
  'Playlist_Id': 'PLuo9105K6NLR0XRJweyZ1vZeqQFiJ-xv8',
  'Published_At': '2024-03-18T06:25:57Z',
  'Video_Count': 10},
 {'Channel_Id': 'UCuI5XcJYynHa5k_lqDzAgwQ',
  'Channel_Name': 'Data Science Tamil',
  'Playlist_Id': 'PLuo9105K6NLRozcPvyXG4fx3z2-cvXG3G',
  'Published_At': '2023-11-16T02:20:12Z',
  'Video_Count': 41},
 {'Channel_Id': 'UCuI5XcJYynHa5k_lqDzAgwQ',
  'Channel_Name': 'Data Science Tamil',
  'Playlist_Id': 'PLuo9105K6NLRe85dP13b0i57AkgyLWOFj',
  'Published_At': '2023-10-17T20:13:20Z',
 

## Transfer Data to MongoDB

In [ ]:
import pymongo as pm

In [ ]:
client=pm.MongoClient("mongodb://localhost:27017")
db=client["YoutubeData"]



In [ ]:
def ImportChannelDetails(channel_id):
    ch_details=get_channel_details(channel_id)
    vid_ids=get_videoIds(channel_id)
    playli_details=PlaylistDetails(channel_id)
    vid_details=get_videoDetails(vid_ids)
    com_details=comments_info(vid_ids)
    
    cl=db['channel_details']
    cl.insert_one({"Channel_Information":ch_details,
                  "Video_Information":vid_details,
                  "Comments_Information":com_details,
                  "Playlist_Information":playli_details})
    return "Uploaded Successfully"

    

In [ ]:

import pandas as pd
channel_data=[]
cl=db['channel_details']
for ch_data in cl.find({},{"_id":0,"Channel_Information":1}):
    channel_data.append(ch_data['Channel_Information'])

df1=pd.DataFrame(channel_data)
df1


,Channel_Name,Channel_Id,Subscription_Count,Channel_Views,Channel_Videos,Channel_Description,Playlist_Id
0,JK,UC_HZY9d5wJ-MEiuq6vhx8hg,887000,104830463,234,🤓\n,UU_HZY9d5wJ-MEiuq6vhx8hg
1,Shree Naval Kishori,UCuMXWW_kAUsW4dC1ZBNGzzQ,205000,16255724,284,,UUuMXWW_kAUsW4dC1ZBNGzzQ
2,Data Science Tamil,UCuI5XcJYynHa5k_lqDzAgwQ,1280,197787,83,If you're looking for more projects or need as...,UUuI5XcJYynHa5k_lqDzAgwQ


## Convert the MongoDB data to SQL

In [ ]:
import psycopg2
def YoutubeChannelDetails():
                                              #Connection with pg andmin SQL
  mydb=psycopg2.connect( host="localhost",
                      database="jithu007",
                        user="postgres",
                        port="5432",
                        password="Indran48@")
  mycursor=mydb.cursor()
  
                                              #Drop table to add more Details
  drop_table='''drop table if exists channels'''
  mycursor.execute(drop_table)
  mydb.commit()
                                              #Create new Table with Same name 
  try:    
    mytable='''Create table if not exists Channels(Channel_Name varchar(50),
                                                    Channel_Id varchar(50) primary key,
                                                    Subscription_Count bigint,
                                                    Channel_Views bigint,
                                                    Channel_Videos bigint,
                                                    Channel_Description text,
                                                    Playlist_Id varchar(50)
                                                )'''
    mycursor.execute(mytable)
    mydb.commit()
  except:
      print("table already created")

                                                #Insert rows into the table
  for index,row in df1.iterrows():
      query='''insert into channels(                  Channel_Name ,
                                                      Channel_Id ,
                                                      Subscription_Count,
                                                      Channel_Views ,
                                                      Channel_Videos ,
                                                      Channel_Description,
                                                      Playlist_Id )
                                          values(%s,%s,%s,%s,%s,%s,%s)'''
      values=(row['Channel_Name'],
              row['Channel_Id'],
              row['Subscription_Count'],
              row['Channel_Views'],
              row['Channel_Videos'],
              row['Channel_Description'],
              row['Playlist_Id'])
      try:
          mycursor.execute(query, values)
          mydb.commit()  # Commit only if successful
      except Exception as e:
          print(f"Error: {e}")
          mydb.rollback()
  return "Uploaded  channel Successfully"


## Playlist upload in the SQL

In [ ]:
import pandas as pd
playlist_data=[]
cl=db['channel_details']
for pl_data in cl.find({},{"_id":0,"Playlist_Information":1}):
    for j in range(len(pl_data['Playlist_Information'])):
        playlist_data.append(pl_data['Playlist_Information'][j])
    
df2=pd.DataFrame(playlist_data)
df2



,Channel_Id,Channel_Name,Playlist_Id,Published_At,Video_Count
0,UC_HZY9d5wJ-MEiuq6vhx8hg,JK,PLKOV1qmFByUTLH4v9grw9SzFEZlI1vYHy,2024-02-15T16:31:25Z,7
1,UC_HZY9d5wJ-MEiuq6vhx8hg,JK,PLKOV1qmFByUSS8OxSdJt5zWHILIxHl_Xv,2023-11-19T17:48:48Z,7
2,UC_HZY9d5wJ-MEiuq6vhx8hg,JK,PLKOV1qmFByUQwpoSbafBUl7VhzgJk3O52,2023-10-31T05:24:54Z,8
3,UC_HZY9d5wJ-MEiuq6vhx8hg,JK,PLKOV1qmFByUQRb9dmzze6tB77Mxucbgzi,2023-05-12T16:16:50Z,7
4,UC_HZY9d5wJ-MEiuq6vhx8hg,JK,PLKOV1qmFByUSnNjUbd4p08nWQQq5zu20b,2023-05-02T15:00:40Z,2
5,UC_HZY9d5wJ-MEiuq6vhx8hg,JK,PLKOV1qmFByUQ5yIsxaDUiLFTkX5HklGbl,2023-04-10T13:00:02Z,3
6,UC_HZY9d5wJ-MEiuq6vhx8hg,JK,PLKOV1qmFByUQfvmVD-vI3YbHhdEIIrZeO,2022-05-06T12:18:18Z,2
7,UC_HZY9d5wJ-MEiuq6vhx8hg,JK,PLKOV1qmFByUQUddx4SEwUNjRXptQSktWU,2021-08-20T09:42:01Z,7
8,UC_HZY9d5wJ-MEiuq6vhx8hg,JK,PLKOV1qmFByUR4TpwNb3Hy80tkOgZGHy7O,2021-02-16T09:37:36Z,6
9,UC_HZY9d5wJ-MEiuq6vhx8hg,JK,PLKOV1qmFByURZugsuimubQTgo6cMCv-SL,2021-01-29T15:42:56Z,32


In [ ]:
def YoutubePlaylistDetails():
                                        #Connection with pg andmin SQL
    mydb=psycopg2.connect( host="localhost",
                        database="jithu007",
                        user="postgres",
                        port="5432",
                        password="Indran48@")
    mycursor=mydb.cursor()

                                                #Drop table to add more Details
    drop_table='''drop table if exists Playlists'''
    mycursor.execute(drop_table)
    mydb.commit()
                                                #Create new Table with Same name 
    try:    
        mytable='''Create table if not exists Playlists(Channel_Id varchar(50),
                                                        Channel_Name varchar(50) ,
                                                        Playlist_Id varchar(50) primary key,
                                                        Published_At timestamp,
                                                        Video_Count bigint
                                                    )'''
        
        mycursor.execute(mytable)
        mydb.commit()
    except:
        print("table already created")


                                                #Insert rows into the table

    for index,row in df2.iterrows():
        query='''insert into Playlists( Channel_Id ,
                                        Channel_Name  ,
                                        Playlist_Id ,
                                        Published_At ,
                                        Video_Count  )

                                        values(%s,%s,%s,%s,%s)'''
        values=(row['Channel_Id'],
                row['Channel_Name'],
                row['Playlist_Id'],
                row['Published_At'],
                row['Video_Count'])
        try:
            mycursor.execute(query, values)
            mydb.commit()  # Commit only if successful
        except Exception as e:
            print(f"Error: {e}")
            mydb.rollback()
    return "Uploaded  playlist Successfully"




## Videos Upload in the SQL

In [ ]:
import pandas as pd
video_data=[]
cl=db['channel_details']
for vi_data in cl.find({},{"_id":0,"Video_Information":1}):
    for j in range(len(vi_data['Video_Information'])):
        video_data.append(vi_data['Video_Information'][j])
    
df3=pd.DataFrame(video_data)
df3

,Channel_Name,Channel_Id,Video_Id,Video_Name,Video_Description,Tags,PublishedAt,View_Count,Like_Count,Dislike_Count,Favorite_Count,Definition,Duration,Thumbnail,Caption_Status
0,JK,UC_HZY9d5wJ-MEiuq6vhx8hg,TmUn_4RmdxA,dogs being dawgs 🐶🎀,wholesome video.mp4\n\n\nIG:\nhttps://www.inst...,"[jk, jk tamil, jk tamil dogs being dawgs, dogs...",2024-11-21T13:58:35Z,259303,30371,None,0,hd,PT11M36S,https://i.ytimg.com/vi/TmUn_4RmdxA/default.jpg,Available
1,JK,UC_HZY9d5wJ-MEiuq6vhx8hg,LN0WCkgIY0Q,WHAT ARE THE ODDS ❗❓,what are the odds.. sollu.. what are the odds....,"[jk tamil, jk, jk latest video, jk tamil lates...",2024-11-09T14:53:12Z,328170,31577,None,0,hd,PT9M,https://i.ytimg.com/vi/LN0WCkgIY0Q/default.jpg,Available
2,JK,UC_HZY9d5wJ-MEiuq6vhx8hg,7VHvqcSOlzM,Thalapathy Vijay's Sarkar ☝ ❗ ❗,"Saar- Car game kuda vilandunga, aana sarkar ga...","[vijay, sarkar infinite, sarkar infinite game,...",2024-11-04T14:12:30Z,415889,57969,None,0,hd,PT8M8S,https://i.ytimg.com/vi/7VHvqcSOlzM/default.jpg,Available
3,JK,UC_HZY9d5wJ-MEiuq6vhx8hg,RYs6Hg8wl2U,First Live Stream 🗣,,None,2024-10-27T16:13:15Z,201804,19385,None,0,hd,PT31M4S,https://i.ytimg.com/vi/RYs6Hg8wl2U/default.jpg,Available
4,JK,UC_HZY9d5wJ-MEiuq6vhx8hg,fIuhG5NoOUk,YOUR BIGGEST FEAR ❗ 😰,Instagram:\nhttps://www.instagram.com/jk16377/,"[fear, fears, jk, jk tamil, meme and answers, ...",2024-10-26T14:14:12Z,342274,38837,None,0,hd,PT9M30S,https://i.ytimg.com/vi/fIuhG5NoOUk/default.jpg,Available
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580,Data Science Tamil,UCuI5XcJYynHa5k_lqDzAgwQ,on-a5-77Y94,YouTube Data Harvesting and Warehousing using ...,Explore our YouTube Data Harvesting and Wareho...,"[youtube data harvesting tamil, youtube data h...",2023-10-26T07:55:52Z,6379,154,None,0,hd,PT45M17S,https://i.ytimg.com/vi/on-a5-77Y94/default.jpg,Available
581,Data Science Tamil,UCuI5XcJYynHa5k_lqDzAgwQ,OM7Bsb9ZCTY,YouTube Data Harvesting and Warehousing using ...,Explore our YouTube Data Harvesting and Wareho...,"[youtube data harvesting tamil, youtube data h...",2023-10-25T14:54:20Z,7937,201,None,0,hd,PT57M20S,https://i.ytimg.com/vi/OM7Bsb9ZCTY/default.jpg,Available
582,Data Science Tamil,UCuI5XcJYynHa5k_lqDzAgwQ,2cyfmA-R-Uw,YouTube Data Harvesting and Warehousing using ...,Explore our YouTube Data Harvesting and Wareho...,"[youtube data harvesting tamil, youtube data h...",2023-10-20T07:59:30Z,10252,183,None,0,hd,PT48M50S,https://i.ytimg.com/vi/2cyfmA-R-Uw/default.jpg,Available
583,Data Science Tamil,UCuI5XcJYynHa5k_lqDzAgwQ,r2sn478BKg4,YouTube Data Harvesting and Warehousing using ...,Explore our YouTube Data Harvesting and Wareho...,"[Data Analysis in Tamil, Tamil Language Data P...",2023-10-18T16:49:15Z,12270,141,None,0,hd,PT42M30S,https://i.ytimg.com/vi/r2sn478BKg4/default.jpg,Available


In [ ]:
def YoutubeVideosDetails():                                           #Connection with pg andmin SQL
    mydb=psycopg2.connect( host="localhost",
                        database="jithu007",
                        user="postgres",
                        port="5432",
                        password="Indran48@")
    mycursor=mydb.cursor()

                                                #Drop table to add more Details
    drop_table='''drop table if exists Videos'''
    mycursor.execute(drop_table)
    mydb.commit()
                                                #Create new Table with Same name 
    try:    
        mytable='''Create table if not exists Videos(Channel_Name varchar(50),
                    Channel_Id varchar(50),
                Video_Id varchar(50) primary key ,
                Video_Name varchar(100),
                Video_Description text,
                Tags text,
                PublishedAt timestamp,
                View_Count bigint,
                Like_Count bigint,
                Dislike_Count bigint ,
                Favorite_Count bigint,
                Definition varchar(100),
                Duration interval,
                Thumbnail  varchar(100),
                Caption_Status varchar(100)
                                                    )'''
        
        mycursor.execute(mytable)
        mydb.commit()
    except:
        print("table already created")


                                                #Insert rows into the table

    for index,row in df3.iterrows():
        query='''insert into Videos(Channel_Name ,
                    Channel_Id ,
                Video_Id ,
                Video_Name ,
                Video_Description ,
                Tags ,
                PublishedAt ,
                View_Count ,
                Like_Count ,
                Dislike_Count  ,
                Favorite_Count ,
                Definition ,
                Duration ,
                Thumbnail  ,
                Caption_Status  )

                                        values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''
        values=(row['Channel_Name'],
                row['Channel_Id'],
                row['Video_Id'],
                row['Video_Name'],
                row['Video_Description'],
                row['Tags'],row['PublishedAt'],row['View_Count'],row['Like_Count'],row['Dislike_Count'],row['Favorite_Count'],row['Definition'],
                row['Duration'],row['Thumbnail'],row['Caption_Status'],)
        try:
            mycursor.execute(query, values)
            mydb.commit()  # Commit only if successful
        except Exception as e:
            print(f"Error: {e}")
            mydb.rollback()
    return "Uploaded  videos Successfully"





## Comments of the Video upload to SQL

In [ ]:
import pandas as pd
cmt_data=[]
cl=db['channel_details']
for cm_data in cl.find({},{"_id":0,"Comments_Information":1}):
    for j in range(len(cm_data['Comments_Information'])):
        cmt_data.append(cm_data['Comments_Information'][j])
    
df4=pd.DataFrame(cmt_data)
df4

,Video_id,Comment_Id,Comment_Text,Comment_Author,Comment_PublishedAt
0,TmUn_4RmdxA,UgwTSDuxJ8a7bndj2IV4AaABAg,Athoda hand vechi nambalaiye vezha veikum cats 😢,@rosenarmatha5487,2024-11-24T02:53:29Z
1,TmUn_4RmdxA,Ugy7tdrX7ap0DLPLJbl4AaABAg,Dogs : உசுர குடுக்கலாம் சார்....<br>Cats : கடம...,@Karthickloveworld,2024-11-24T02:16:33Z
2,TmUn_4RmdxA,UgxuNjY75L9gbq4ia8l4AaABAg,Enoda car en bed ley thungu bathroom pora apo ...,@ELANGOK-bz6ln,2024-11-23T21:39:29Z
3,TmUn_4RmdxA,UgxSN67QGqDpLI6L8Yh4AaABAg,Andha owner mela onnuku aducha dog uh fake and...,@MevinKumar,2024-11-23T21:09:00Z
4,TmUn_4RmdxA,Ugy5umbsrwzwtYlgmmF4AaABAg,bro casually skipped over the reel where the d...,@Akth1518,2024-11-23T19:41:56Z
...,...,...,...,...,...
37689,xBCx4PEZpfo,UgxR4MTiscRFc-1qIad4AaABAg,couldnt create environment bro pls assist,@PriyankaRangaswamy-no2iv,2023-11-14T13:48:46Z
37690,xBCx4PEZpfo,UgyOU6rMsn1BtZmYbjN4AaABAg,bro jupyter laium streamlit install pananuma,@Vignesh-l5z,2023-11-06T10:43:12Z
37691,xBCx4PEZpfo,UgzKpTwOasD2wXZsLQh4AaABAg,Hi bro please share the instgram or LinkedIn l...,@UDSARAVINDAGOKULP,2023-11-04T14:44:33Z
37692,xBCx4PEZpfo,Ugz-b9_-vpME9qwKkj14AaABAg,Are u currently at DS course?,@revanthragult4803,2023-11-03T01:12:58Z


In [ ]:
def YoutubeCommentsDetails():                                         #Connection with pg andmin SQL
    mydb=psycopg2.connect( host="localhost",
                        database="jithu007",
                        user="postgres",
                        port="5432",
                        password="Indran48@")
    mycursor=mydb.cursor()

                                                #Drop table to add more Details
    drop_table='''drop table if exists Comments'''
    mycursor.execute(drop_table)
    mydb.commit()
                                                #Create new Table with Same name 
    try:    
        mytable='''Create table if not exists Comments( Video_id varchar(50) ,
                    Comment_Id varchar(100) primary key ,
                Comment_Text text ,
                Comment_Author varchar(100),
                Comment_PublishedAt timestamp
                                                    )'''
        
        mycursor.execute(mytable)
        mydb.commit()
    except:
        print("table already created")


                                                #Insert rows into the table

    for index,row in df4.iterrows():
        query='''insert into Comments(  Video_id ,
                    Comment_Id ,
                Comment_Text  ,
                Comment_Author ,
                Comment_PublishedAt  )

                                        values(%s,%s,%s,%s,%s)'''
        values=(row['Video_id'],
                row['Comment_Id'],
                row['Comment_Text'],
                row['Comment_Author'],
                row['Comment_PublishedAt'])
        try:
            mycursor.execute(query, values)
            mydb.commit()  # Commit only if successful
        except Exception as e:
            print(f"Error: {e}")
            mydb.rollback()
    return "Uploaded  comments Successfully"


In [ ]:
def tables_SQL():
    YoutubeChannelDetails()
    YoutubePlaylistDetails()
    YoutubeVideosDetails()
    YoutubeCommentsDetails()
    return " tables Created"



In [ ]:
Tables=tables_SQL()

In [ ]:
def show_channel_details():
   
    channel_data=[]
    cl=db['channel_details']
    for ch_data in cl.find({},{"_id":0,"Channel_Information":1}):
        channel_data.append(ch_data['Channel_Information'])

    df1=st.dataframe(channel_data)
    df1
    return channel_data

2024-11-24 12:54:18.658 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-24 12:54:18.660 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [ ]:
def show_playlists_details():
    playlist_data=[]
    cl=db['channel_details']
    for pl_data in cl.find({},{"_id":0,"Playlist_Information":1}):
        for j in range(len(pl_data['Playlist_Information'])):
            playlist_data.append(pl_data['Playlist_Information'][j])
        
    df2=st.dataframe(playlist_data)
    
    return playlist_data

In [ ]:
def show_videos_details():

    video_data=[]
    cl=db['channel_details']
    for vi_data in cl.find({},{"_id":0,"Video_Information":1}):
        for j in range(len(vi_data['Video_Information'])):
            video_data.append(vi_data['Video_Information'][j])
        
    df3=st.dataframe(video_data)
    
    return video_data

In [ ]:
def show_comments_details():

    cmt_data=[]
    cl=db['channel_details']
    for cm_data in cl.find({},{"_id":0,"Comments_Information":1}):
        for j in range(len(cm_data['Comments_Information'])):
            cmt_data.append(cm_data['Comments_Information'][j])
        
    df4=st.dataframe(cmt_data)
    df4
    return cmt_data

2024-11-24 12:56:53.438 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-24 12:56:53.441 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

## Stream Lit Codings

In [ ]:
with st.sidebar:
    st.title(":blue[Youtube Harvesting and Warehouse]")
    st.header(":red"["Skill Enhancement"])
    st.caption('Python Scripting')
    st.caption('Data Collection')
    st.caption('MongoDB')
    st.caption('API Integration')
    st.caption('Data Management using MongoDB & SQL')

channel_id=st.text_input("Enter the Channel ID")
if st.button("Collect & Store Data"):
    chl_ID=[]
    db=client["YoutubeData"]
    cl=db['channel_details']
    for ch_data in cl.find({},{"_id":0,"Channel_Information":1}):
        chl_ID.append(ch_data['Channel_Information']['Channel_Id'])
    if channel_id in chl_ID:
        st.success("Channel_id Already Exists")
    else:
        Insert=ImportChannelDetails(channel_id)
        st.success("Data Inserted")
if st.button("Transfer to SQL"):
    Tables=tables_SQL()
    st.success(Tables)

show_table=st.radio("SELECT THE TABLE",("CHANNELS","PLAYLISTS","VIDEOS","cOMMENTS"))

if show_table=="CHANNELS":
    show_channel_details()
if show_table=="PLAYLISTS":
    show_playlists_details()
if show_table=="VIDEOS":
    show_videos_details()
if show_table=="COMMENTS":
    show_comments_details()


    
    
    


In [ ]:

Questions=st.selectbox(("Select Your Question"),("1.What are the names of all the videos and their corresponding channels?"
                    "2.Which channels have the most number of videos, and how many videos do  they have?"
                    "3.What are the top 10 most viewed videos and their respective channels?"
                    "4.How many comments were made on each video, and what are their  corresponding video names?"
                    "5.Which videos have the highest number of likes, and what are their corresponding channel names?"
                    "6.What is the total number of likes and dislikes for each video, and what are their corresponding video names?"
                    "7.What is the total number of views for each channel, and what are their corresponding channel names?"
                    "8.What are the names of all the channels that have published videos in the year  2022?"
                    "9.What is the average duration of all videos in each channel, and what are their corresponding channel names?"
                    "10.Which videos have the highest number of comments, and what are their corresponding channel names?"
))
     

2024-11-24 13:48:50.330 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-24 13:48:50.332 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-24 13:48:50.333 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-24 13:48:50.334 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-24 13:48:50.335 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-24 13:48:50.336 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [ ]:
mydb=psycopg2.connect( host="localhost",        
                    database="jithu007",
                    user="postgres",
                    port="5432",
                    password="Indran48@")     #Connection with pg andmin SQL
mycursor=mydb.cursor()


Questions=st.selectbox(("Select Your Question"),("1.What are the names of all the videos and their corresponding channels?"
                    "2.Which channels have the most number of videos, and how many videos do  they have?"
                    "3.What are the top 10 most viewed videos and their respective channels?"
                    "4.How many comments were made on each video, and what are their  corresponding video names?"
                    "5.Which videos have the highest number of likes, and what are their corresponding channel names?"
                    "6.What is the total number of likes and dislikes for each video, and what are their corresponding video names?"
                    "7.What is the total number of views for each channel, and what are their corresponding channel names?"
                    "8.What are the names of all the channels that have published videos in the year  2022?"
                    "9.What is the average duration of all videos in each channel, and what are their corresponding channel names?"
                    "10.Which videos have the highest number of comments, and what are their corresponding channel names?"
))

if Questions=="1.What are the names of all the videos and their corresponding channels?":
    query1="select channel_name as Name_of_the_Channel,video_name as Name_of_the_Video from videos  order by Name_of_the_Channel asc "
    mycursor.execute(query1)
    mydb.commit()
    t1=mycursor.fetchall()
    dfa=pd.DataFrame(t1,columns=["Channel_name","No.of.Videos"])


NameError: name 'psycopg2' is not defined

In [ ]:
if Questions=="1.What are the names of all the videos and their corresponding channels?":
    query1="select channel_name as Name_of_the_Channel,video_name as Name_of_the_Video from videos  order by Name_of_the_Channel asc "
    mycursor.execute(query1)
    mydb.commit()
    t1=mycursor.fetchall()
    dfa=pd.DataFrame(t1,columns=["Channel_name","No.of.Videos"])
    st.write(dfa)

NameError: name 'Questions' is not defined